# Reconciliation #02: Cash Flow Statement Reconciliation

## Overview
- **What this notebook tests:** The cash flow statement reconciles correctly: `Beginning Cash + Net Change in Cash == Ending Cash` (no plug figures), and the three-section subtotals (`Operating + Investing + Financing`) compose cleanly into the net change.
- **Prerequisites:** `pip install -e .` from the project root.
- **Estimated runtime:** < 60 seconds
- **Audience:** Developers, Actuaries, CFOs, Auditors

## Why This Matters
A cash flow statement that silently "plugs" the net change to match the actual cash delta hides calculation errors. This notebook verifies that:

1. **Internal consistency** -- `Operating + Investing + Financing == Net Change in Cash` (three-section composition).
2. **No plug regression** -- With controlled metrics (known cash flows), the calculated net change equals the actual cash change, confirming no silent overwrite (Issue #240).
3. **Simulation-based reconciliation** -- Running a real manufacturer and comparing the indirect method against the direct (ledger-sourced) method.
4. **Multi-year traceability** -- Operating CF components trace back to income statement items and working capital changes.

Part of the reconciliation smoke-test suite ([Issue #1393](https://github.com/AlexFiliakov/Ergodic-Insurance-Limits/issues/1393)).

## Setup

In [ ]:
# Standard library
import sys
import os
import random
import warnings
from decimal import Decimal

# Third-party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Reconciliation helpers (shared across all reconciliation notebooks)
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath(".")), "reconciliation"))
sys.path.insert(0, os.path.abspath("."))

from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df,
    create_standard_manufacturer
)

# Core framework
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.financial_statements import CashFlowStatement

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Suppress noisy warnings during simulation
warnings.filterwarnings("ignore", category=UserWarning)

print("All imports successful.")

In [ ]:
notebook_header(
    number=2,
    title="Cash Flow Statement Reconciliation",
    description=(
        "Verifies cash flow statement internal consistency, no-plug regression, "
        "and direct vs. indirect method agreement."
    ),
)

---
## Section 1: No-Plug Regression Test (Controlled Metrics)

This is the most critical test. We construct metrics where the cash flow components are
**pre-calculated** to be consistent with the ending cash. This verifies that the
`CashFlowStatement` class reports the *calculated* net change, not a plugged value
that silently overwrites discrepancies (Issue #240).

**Test design:** We define two years of metrics where:
- Operating CF = Net Income + Depreciation + Working Capital Changes
- Investing CF = -(Change in Net PP&E + Depreciation)
- Financing CF = -Dividends
- Ending Cash = Beginning Cash + (Operating + Investing + Financing)

In [ ]:
section_header("1. No-Plug Regression Test (Controlled Metrics)")

checker_plug = ReconciliationChecker(section="No-Plug Regression (Controlled Metrics)")

with timed_cell("Controlled metrics reconciliation"):
    # Scenario A: Simple case (no working capital changes)
    # Operating: 600k NI + 60k Dep = 660k
    # Investing: -(40k net PPE chg + 60k Dep) = -100k
    # Financing: -180k dividends
    # Net: 660k - 100k - 180k = 380k
    # Ending: 1,000k + 380k = 1,380k
    metrics_simple = [
        {
            "cash": 1_000_000,
            "net_income": 500_000,
            "dividends_paid": 150_000,
            "depreciation_expense": 50_000,
            "gross_ppe": 500_000,
            "net_ppe": 450_000,
        },
        {
            "cash": 1_380_000,
            "net_income": 600_000,
            "dividends_paid": 180_000,
            "depreciation_expense": 60_000,
            "gross_ppe": 600_000,
            "net_ppe": 490_000,
        },
    ]

    cfs_simple = CashFlowStatement(metrics_simple)
    df_simple = cfs_simple.generate_statement(1)
    col = "Year 1"

    vals_a = {}
    for _, row in df_simple.iterrows():
        item = row["Item"].strip()
        val = row[col]
        if val != "" and isinstance(val, (int, float, Decimal)):
            vals_a[item] = float(val)

    begin_a = vals_a.get("Cash - Beginning of Period", 0)
    net_change_a = vals_a.get("Net Change in Cash", 0)
    ending_a = vals_a.get("Cash - End of Period", 0)
    net_inc_a = vals_a.get("NET INCREASE (DECREASE) IN CASH", 0)

    checker_plug.assert_close(
        actual=begin_a + net_change_a,
        expected=ending_a,
        tol=0.01,
        message="Scenario A (simple): Beginning + Net Change == Ending",
    )
    checker_plug.assert_close(
        actual=net_inc_a, expected=380_000,
        tol=0.01,
        message="Scenario A: Calculated net change == 380,000 (no plug)",
    )
    checker_plug.assert_close(
        actual=begin_a, expected=1_000_000, tol=0.01,
        message="Scenario A: Beginning cash == 1,000,000",
    )
    checker_plug.assert_close(
        actual=ending_a, expected=1_380_000, tol=0.01,
        message="Scenario A: Ending cash == 1,380,000",
    )

    # Scenario B: With full working capital changes
    # Operating: 600k NI + 110k Dep - 50k AR - 50k Inv - 10k Prepaid + 30k AP + 20k Accrued + 50k Claims = 700k
    # Investing: -(190k net PPE chg + 110k Dep) = -300k
    # Financing: -180k dividends
    # Net: 700k - 300k - 180k = 220k
    # Ending: 1,000k + 220k = 1,220k
    metrics_wc = [
        {
            "cash": 1_000_000,
            "net_income": 500_000,
            "depreciation_expense": 100_000,
            "accounts_receivable": 200_000,
            "inventory": 150_000,
            "prepaid_insurance": 20_000,
            "accounts_payable": 100_000,
            "accrued_expenses": 50_000,
            "claim_liabilities": 30_000,
            "gross_ppe": 1_000_000,
            "net_ppe": 900_000,
            "dividends_paid": 150_000,
        },
        {
            "cash": 1_220_000,
            "net_income": 600_000,
            "depreciation_expense": 110_000,
            "accounts_receivable": 250_000,
            "inventory": 200_000,
            "prepaid_insurance": 30_000,
            "accounts_payable": 130_000,
            "accrued_expenses": 70_000,
            "claim_liabilities": 80_000,
            "gross_ppe": 1_300_000,
            "net_ppe": 1_090_000,
            "dividends_paid": 180_000,
        },
    ]

    cfs_wc = CashFlowStatement(metrics_wc)
    df_wc = cfs_wc.generate_statement(1)

    vals_b = {}
    for _, row in df_wc.iterrows():
        item = row["Item"].strip()
        val = row[col]
        if val != "" and isinstance(val, (int, float, Decimal)):
            vals_b[item] = float(val)

    begin_b = vals_b.get("Cash - Beginning of Period", 0)
    net_change_b = vals_b.get("Net Change in Cash", 0)
    ending_b = vals_b.get("Cash - End of Period", 0)
    net_inc_b = vals_b.get("NET INCREASE (DECREASE) IN CASH", 0)

    checker_plug.assert_close(
        actual=begin_b + net_change_b,
        expected=ending_b,
        tol=0.01,
        message="Scenario B (with WC): Beginning + Net Change == Ending",
    )
    checker_plug.assert_close(
        actual=net_inc_b, expected=220_000,
        tol=0.01,
        message="Scenario B: Calculated net change == 220,000 (no plug)",
    )

    # Scenario C: Negative cash flow (net loss year)
    # Operating: -200k NI + 50k Dep = -150k
    # Investing: 0 (no PPE change)
    # Financing: 0 (no dividends on loss)
    # Net: -150k
    # Ending: 1,000k - 150k = 850k
    metrics_loss = [
        {
            "cash": 1_000_000,
            "net_income": 100_000,
            "depreciation_expense": 50_000,
            "gross_ppe": 1_000_000,
            "net_ppe": 950_000,
        },
        {
            "cash": 850_000,
            "net_income": -200_000,
            "depreciation_expense": 50_000,
            "gross_ppe": 1_000_000,
            "net_ppe": 900_000,
            "dividends_paid": 0,
        },
    ]

    cfs_loss = CashFlowStatement(metrics_loss)
    df_loss = cfs_loss.generate_statement(1)

    vals_c = {}
    for _, row in df_loss.iterrows():
        item = row["Item"].strip()
        val = row[col]
        if val != "" and isinstance(val, (int, float, Decimal)):
            vals_c[item] = float(val)

    begin_c = vals_c.get("Cash - Beginning of Period", 0)
    net_change_c = vals_c.get("Net Change in Cash", 0)
    ending_c = vals_c.get("Cash - End of Period", 0)
    net_inc_c = vals_c.get("NET INCREASE (DECREASE) IN CASH", 0)

    checker_plug.assert_close(
        actual=begin_c + net_change_c,
        expected=ending_c,
        tol=0.01,
        message="Scenario C (loss year): Beginning + Net Change == Ending",
    )
    checker_plug.assert_close(
        actual=net_inc_c, expected=-150_000,
        tol=0.01,
        message="Scenario C: Calculated net change == -150,000",
    )

checker_plug.display_results()

---
## Section 2: Three-Section Composition Check

For every scenario, verify that the three cash flow sections compose correctly:
`Operating + Investing + Financing == NET INCREASE (DECREASE) IN CASH`

In [ ]:
section_header("2. Three-Section Composition")

checker_compose = ReconciliationChecker(section="Three-Section Composition")

scenarios = [
    ("Simple", cfs_simple, 1),
    ("Working Capital", cfs_wc, 1),
    ("Loss Year", cfs_loss, 1),
]

for label, cfs_obj, yr in scenarios:
    df = cfs_obj.generate_statement(yr)
    col = f"Year {yr}"

    vals = {}
    for _, row in df.iterrows():
        item = row["Item"].strip()
        val = row[col]
        if val != "" and isinstance(val, (int, float, Decimal)):
            vals[item] = float(val)

    operating = vals.get("Net Cash Provided by Operating Activities", 0)
    investing = vals.get("Net Cash Used in Investing Activities", 0)
    financing = vals.get("Net Cash Used in Financing Activities", 0)
    net_change = vals.get("NET INCREASE (DECREASE) IN CASH", 0)

    sum_sections = operating + investing + financing

    checker_compose.assert_close(
        actual=sum_sections,
        expected=net_change,
        tol=0.01,
        message=f"{label}: Oper ({fmt_dollar(operating)}) + Inv ({fmt_dollar(investing)}) + Fin ({fmt_dollar(financing)}) == Net ({fmt_dollar(net_change)})",
        label_actual="Sum of Sections",
        label_expected="Net Change",
    )

checker_compose.display_results()

---
## Section 3: Manufacturer Simulation -- Multi-Year Cash Flow

Now we create a deterministic manufacturer ($10M assets, 10% margin, 25% tax) and run a multi-year simulation, collecting metrics each year. We generate cash flow statements and verify internal consistency.

In [ ]:
section_header("3. Manufacturer Simulation")

NUM_YEARS = 5

with timed_cell("Manufacturer creation and simulation"):
    manufacturer = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    # Collect year-0 (initial) metrics before any step
    all_metrics = []
    initial_metrics = manufacturer.calculate_metrics()
    initial_metrics["year"] = 0
    all_metrics.append(initial_metrics)

    for yr in range(NUM_YEARS):
        step_metrics = manufacturer.step()
        all_metrics.append(step_metrics)

    print(f"Simulation complete: {NUM_YEARS} years")
    print(f"Metrics collected: {len(all_metrics)} periods (year 0 initial + {NUM_YEARS} steps)")

    # Summary table
    summary_rows = []
    for i, m in enumerate(all_metrics):
        summary_rows.append({
            "Period": i,
            "Cash": fmt_dollar(float(m['cash'])),
            "Net Income": fmt_dollar(float(m['net_income'])),
            "Depreciation": fmt_dollar(float(m['depreciation_expense'])),
            "Dividends Paid": fmt_dollar(float(m.get('dividends_paid', 0))),
            "Equity": fmt_dollar(float(m['equity'])),
        })
    display_df(pd.DataFrame(summary_rows), title="Simulation Metrics Summary")

---
## Section 4: Indirect Method -- Internal Consistency

For each simulation year, generate a cash flow statement (indirect method) and verify:
1. `Operating + Investing + Financing == NET INCREASE (DECREASE) IN CASH` (three-section composition)
2. Net income in the statement matches the metrics
3. Depreciation add-back matches the metrics

In [ ]:
section_header("4. Indirect Method -- Internal Consistency")

checker_indirect = ReconciliationChecker(section="Indirect Method Internal Consistency")

with timed_cell("Generate indirect-method statements"):
    cfs = CashFlowStatement(all_metrics)

    recon_rows = []
    for yr in range(1, len(all_metrics)):
        df = cfs.generate_statement(yr)
        col_name = f"Year {yr}"

        vals = {}
        for _, row in df.iterrows():
            item = row["Item"].strip()
            val = row[col_name]
            if val != "" and isinstance(val, (int, float, Decimal)):
                vals[item] = float(val)

        operating = vals.get("Net Cash Provided by Operating Activities", 0)
        investing = vals.get("Net Cash Used in Investing Activities", 0)
        financing = vals.get("Net Cash Used in Financing Activities", 0)
        net_increase = vals.get("NET INCREASE (DECREASE) IN CASH", 0)
        beginning = vals.get("Cash - Beginning of Period", 0)
        net_change = vals.get("Net Change in Cash", 0)
        ending = vals.get("Cash - End of Period", 0)
        ni_stmt = vals.get("Net Income", 0)
        dep_stmt = vals.get("Depreciation and Amortization", 0)

        # Check 1: Three-section composition
        checker_indirect.assert_close(
            actual=operating + investing + financing,
            expected=net_increase,
            tol=0.01,
            message=f"Year {yr}: Oper + Inv + Fin == Net Increase",
            label_actual="Sum of Sections",
            label_expected="Net Increase",
        )

        # Check 2: Net income matches metrics
        checker_indirect.assert_close(
            actual=ni_stmt,
            expected=float(all_metrics[yr].get("net_income", 0)),
            tol=0.01,
            message=f"Year {yr}: Statement NI matches metrics NI",
            label_actual="Statement NI",
            label_expected="Metrics NI",
        )

        # Check 3: Depreciation matches metrics
        checker_indirect.assert_close(
            actual=dep_stmt,
            expected=float(all_metrics[yr].get("depreciation_expense", 0)),
            tol=0.01,
            message=f"Year {yr}: Statement Depreciation matches metrics",
            label_actual="Statement Dep",
            label_expected="Metrics Dep",
        )

        # Check 4: Beginning cash matches prior period ending cash from metrics
        prior_cash = float(all_metrics[yr - 1].get("cash", 0))
        checker_indirect.assert_close(
            actual=beginning,
            expected=prior_cash,
            tol=0.01,
            message=f"Year {yr}: Beginning cash == prior period cash from metrics",
            label_actual="Statement Beginning",
            label_expected="Prior Metrics Cash",
        )

        # Check 5: Ending cash matches current period cash from metrics
        current_cash = float(all_metrics[yr].get("cash", 0))
        checker_indirect.assert_close(
            actual=ending,
            expected=current_cash,
            tol=0.01,
            message=f"Year {yr}: Ending cash == current period cash from metrics",
            label_actual="Statement Ending",
            label_expected="Metrics Cash",
        )

        recon_rows.append({
            "Year": yr,
            "Beginning Cash": beginning,
            "Operating CF": operating,
            "Investing CF": investing,
            "Financing CF": financing,
            "Net Change": net_change,
            "Ending Cash": ending,
            "Actual Cash Delta": current_cash - prior_cash,
        })

    recon_df = pd.DataFrame(recon_rows)
    fmt_recon = recon_df.copy()
    for c in fmt_recon.columns[1:]:
        fmt_recon[c] = fmt_recon[c].apply(lambda x: f"${x:,.0f}")
    display_df(fmt_recon, title="Cash Flow Summary (Indirect Method)")

checker_indirect.display_results()

---
## Section 5: Direct vs. Indirect Method -- Net Cash Flow Agreement

When the manufacturer's ledger is available, we generate cash flow statements using the
**direct method** (summing actual cash transactions from the ledger). Both methods should
report the same beginning cash, ending cash, and internally consistent totals.

In [ ]:
section_header("5. Direct vs. Indirect Method Comparison")

checker_methods = ReconciliationChecker(section="Direct vs. Indirect Method")

with timed_cell("Generate direct-method statements"):
    cfs_direct = CashFlowStatement(all_metrics, ledger=manufacturer.ledger)

    comparison_rows = []

    for yr in range(1, len(all_metrics)):
        # Generate both statements
        df_indirect = cfs.generate_statement(yr, method="indirect")
        df_direct = cfs_direct.generate_statement(yr, method="direct")
        col_name = f"Year {yr}"

        # Extract indirect values
        indirect_vals = {}
        for _, row in df_indirect.iterrows():
            item = row["Item"].strip()
            val = row[col_name]
            if val != "" and isinstance(val, (int, float, Decimal)):
                indirect_vals[item] = float(val)

        # Extract direct values
        direct_vals = {}
        for _, row in df_direct.iterrows():
            item = row["Item"].strip()
            val = row[col_name]
            if val != "" and isinstance(val, (int, float, Decimal)):
                direct_vals[item] = float(val)

        # Both methods should report same beginning and ending cash
        i_begin = indirect_vals.get("Cash - Beginning of Period", 0)
        d_begin = direct_vals.get("Cash - Beginning of Period", 0)
        i_end = indirect_vals.get("Cash - End of Period", 0)
        d_end = direct_vals.get("Cash - End of Period", 0)

        checker_methods.assert_close(
            actual=d_begin, expected=i_begin, tol=0.01,
            message=f"Year {yr}: Direct beginning cash == Indirect beginning cash",
            label_actual="Direct Begin",
            label_expected="Indirect Begin",
        )
        checker_methods.assert_close(
            actual=d_end, expected=i_end, tol=0.01,
            message=f"Year {yr}: Direct ending cash == Indirect ending cash",
            label_actual="Direct End",
            label_expected="Indirect End",
        )

        # Direct method should be internally consistent:
        # its own Operating + Investing + Financing == its own Net Increase
        d_operating = direct_vals.get("Net Cash Provided by Operating Activities", 0)
        d_investing = direct_vals.get("Net Cash Used in Investing Activities", 0)
        d_financing = direct_vals.get("Net Cash Used in Financing Activities", 0)
        d_net_inc = direct_vals.get("NET INCREASE (DECREASE) IN CASH", 0)

        checker_methods.assert_close(
            actual=d_operating + d_investing + d_financing,
            expected=d_net_inc,
            tol=0.01,
            message=f"Year {yr}: Direct method three-section composition",
            label_actual="Sum of Sections",
            label_expected="Net Increase",
        )

        # Get indirect totals for comparison table
        i_net_inc = indirect_vals.get("NET INCREASE (DECREASE) IN CASH", 0)

        comparison_rows.append({
            "Year": yr,
            "Indirect Net": i_net_inc,
            "Direct Net": d_net_inc,
            "Net Delta": abs(i_net_inc - d_net_inc),
            "Actual Cash Delta": float(all_metrics[yr]['cash']) - float(all_metrics[yr - 1]['cash']),
        })

    comp_df = pd.DataFrame(comparison_rows)
    fmt_comp = comp_df.copy()
    for c in fmt_comp.columns[1:]:
        fmt_comp[c] = fmt_comp[c].apply(lambda x: f"${x:,.0f}")
    display_df(fmt_comp, title="Direct vs. Indirect Net Cash Flow")

checker_methods.display_results()

---
## Section 6: Waterfall Chart -- Cash Flow Components

Visualize the cash flow waterfall for representative years using the controlled (no-plug) scenarios:
Beginning Cash -> Operating -> Investing -> Financing -> Ending Cash.

In [ ]:
section_header("6. Cash Flow Waterfall Charts")

# Build waterfall data from the controlled scenarios
waterfall_scenarios = [
    ("Scenario A: Simple", vals_a, 380_000),
    ("Scenario B: With WC", vals_b, 220_000),
    ("Scenario C: Loss Year", vals_c, -150_000),
]

fig, axes = plt.subplots(1, 3, figsize=(18, 5), squeeze=False)
axes = axes.flatten()

for idx, (title, vals, expected_net) in enumerate(waterfall_scenarios):
    ax = axes[idx]

    beginning = vals.get("Cash - Beginning of Period", 0)
    op_cf = vals.get("Net Cash Provided by Operating Activities", 0)
    inv_cf = vals.get("Net Cash Used in Investing Activities", 0)
    fin_cf = vals.get("Net Cash Used in Financing Activities", 0)
    ending = vals.get("Cash - End of Period", 0)
    net_change = vals.get("Net Change in Cash", 0)
    delta = abs((beginning + net_change) - ending)

    labels = ["Beginning\nCash", "Operating", "Investing", "Financing", "Ending\nCash"]
    flow_vals = [beginning, op_cf, inv_cf, fin_cf, ending]

    colors = []
    bottoms = []
    bar_heights = []
    running = 0

    for i, (lbl, val) in enumerate(zip(labels, flow_vals)):
        if i == 0:  # Beginning cash
            bottoms.append(0)
            bar_heights.append(val)
            colors.append("#4a86c8")
            running = val
        elif i == 4:  # Ending cash
            bottoms.append(0)
            bar_heights.append(val)
            colors.append("#4a86c8")
        else:  # Flow items
            if val >= 0:
                bottoms.append(running)
                bar_heights.append(val)
                colors.append("#28a745")
            else:
                bottoms.append(running + val)
                bar_heights.append(abs(val))
                colors.append("#dc3545")
            running += val

    ax.bar(labels, bar_heights, bottom=bottoms, color=colors, edgecolor="white", linewidth=0.5)
    ax.set_title(title, fontsize=11, fontweight="bold")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"${x/1e6:.1f}M" if abs(x) >= 1e6 else f"${x/1e3:.0f}K"))
    ax.tick_params(axis="x", labelsize=8)
    ax.grid(axis="y", alpha=0.3)

    # Annotate delta
    ax.annotate(
        f"Delta: ${delta:.2f}",
        xy=(0.98, 0.02), xycoords="axes fraction",
        ha="right", va="bottom", fontsize=8,
        color="#28a745" if delta < 0.01 else "#dc3545",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

print("Green bars = cash inflows, Red bars = cash outflows, Blue bars = balance positions.")
print("Delta annotations confirm zero discrepancy in controlled scenarios.")

---
## Section 7: Tolerance Breach Report

Explicitly check whether any controlled scenario has a reconciliation delta exceeding the $0.01 tolerance.

In [ ]:
section_header("7. Tolerance Breach Report")

checker_tolerance = ReconciliationChecker(section="Tolerance Breach Check")

TOLERANCE = 0.01

controlled_results = [
    ("Scenario A (simple)", vals_a),
    ("Scenario B (working capital)", vals_b),
    ("Scenario C (loss year)", vals_c),
]

breach_count = 0
for label, vals in controlled_results:
    beginning = vals.get("Cash - Beginning of Period", 0)
    net_change = vals.get("Net Change in Cash", 0)
    ending = vals.get("Cash - End of Period", 0)
    delta = abs((beginning + net_change) - ending)

    passed = checker_tolerance.check(
        condition=(delta <= TOLERANCE),
        message=f"{label}: Delta ${delta:.6f} within ${TOLERANCE:.2f} tolerance",
        detail=f"Beginning={fmt_dollar(beginning)}, Net Change={fmt_dollar(net_change)}, Ending={fmt_dollar(ending)}",
    )
    if not passed:
        breach_count += 1

if breach_count == 0:
    print(f"No tolerance breaches detected across {len(controlled_results)} controlled scenarios.")
else:
    print(f"WARNING: {breach_count} scenario(s) exceeded the ${TOLERANCE:.2f} tolerance!")

checker_tolerance.display_results()

---
## Section 8: Detailed Cash Flow Statement Display

Display a full cash flow statement from the controlled working-capital scenario as a reference.

In [ ]:
section_header("8. Sample Full Cash Flow Statement (Controlled WC Scenario)")

df_display = cfs_wc.generate_statement(1)
col = "Year 1"

def format_cf_value(val):
    """Format a cash flow statement value for display."""
    if isinstance(val, (int, float, Decimal)) and val != "":
        v = float(val)
        if v == 0:
            return "-"
        return f"${v:,.0f}"
    return str(val)

display_version = df_display[["Item", col]].copy()
display_version[col] = display_version[col].apply(format_cf_value)
display_df(display_version, title="Cash Flow Statement -- Year 1 (Indirect Method, Full WC)")

---
## Final Summary

Aggregate all checks from all sections and produce the final PASS/FAIL verdict.

In [ ]:
final_summary(checker_plug, checker_compose, checker_indirect, checker_methods, checker_tolerance)